# Load Libraries

In [33]:
import xml.etree.ElementTree as ET
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
from sklearn import preprocessing

import zipfile
from glob import glob

import seaborn as sns
from sqlalchemy import column
from sympy import subsets
sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

#import japanize_matplotlib
import unicodedata
import mojimoji

from MyZipClass import MyZipClass
from NIKKEIHokenClass import NIKKEIHokenClass
from MyKanjyaClass import MyKanjyaClass

## Class definition

In [34]:
class HokenCheckClass(object):

    def __init__(self):

        self.data_dir = u"S:\\"
        self.output_dir = u"L:\\epson_pxm840f\\Shikakku"

        self.shikaku_tag = {}        
        self.shikaku = []

        # expand latest shikaku history file
        zipObject = MyZipClass()
        self.df_shikaku = zipObject.readZipFile()
        #zipObject.toCsv(self.df_shikaku,"資格確認_保険証.csv")

        # read NIKKEI data to hold Domestic/Social Insurance
        nikkeiObject = NIKKEIHokenClass()
        self.df_nikkei = nikkeiObject.readShikaku()
        #nikkeiObject.toCsv(self.df_nikkei, filename="NIKKEI_hoken.csv")

        kanjyaObject = MyKanjyaClass()
        self.df_kanjya = kanjyaObject.readCSV()
        #kanjyaObject.toCsv(self.df_kanjya,"kanjya.csv")

    def merge(self):
        selected=['id','Name','birth','InsurerSegment','InsurerNumber',
                    'InsuredCardSymbol','InsuredIdentificationNumber',
                    'kouhi1','kouhi_jyu1','kouhi2','kouhi_jyu2',
                    'kyufu','institution']

        df_nikkei = self.df_nikkei[selected].copy()
        df_nikkei = df_nikkei[~df_nikkei.duplicated()].copy()
        # merged with Knajya database on id
        df_nikkei = pd.merge(df_nikkei, self.df_kanjya, on=["id"], how="left")

        #print(df_nikkei.columns)
        #df_nikkei = df_nikkei[df_nikkei.id > 90009358].sort_values('id', ascending=False).copy()
        #selectednew=['chozai','id','Name_x','birth_x','InsurerSegment','InsurerNumber',
        #            'InsuredCardSymbol','InsuredIdentificationNumber',
        #            'kouhi1','kouhi_jyu1','kouhi2','kouhi_jyu2',
        #            'kyufu','institution']
        #df_nikkei[selectednew].columns = selected
        #print(df_nikkei.head(10))
        df_merged = pd.merge(df_nikkei,self.df_shikaku,on=["birth", "Name"], how="left")        
        df_merged_birth_hoken = pd.merge(df_nikkei,self.df_shikaku,on=["birth", "InsurerNumber", 
                                'InsuredCardSymbol','InsuredIdentificationNumber'] )

        self.df_merged_birth_hoken2 = pd.merge(df_nikkei,self.df_shikaku,on=["birth", "InsurerNumber", 
                                'InsuredCardSymbol','InsuredIdentificationNumber'], how="left" )



                # 保険番号、記号、番号、性別　確認
        masks = (df_merged.InsurerNumber_x == df_merged.InsurerNumber_y)  &  \
            (df_merged.InsuredCardSymbol_x == df_merged.InsuredCardSymbol_y )  &   \
             (df_merged.InsuredIdentificationNumber_x == df_merged.InsuredIdentificationNumber_y) # & \
                # (df_merged.sex1 == df_merged.sex2)


        self.test_df = df_merged[masks]

    def dftoCsv(self,df, filename):
        super().toCsv(df, filename)
        #filename = os.path.join(self.output_dir,filename)
        #df.to_csv(filename, index=False, encoding='cp932', errors='replace')

    def mergeCheck(self):
        
        selected=['id','Name','birth','InsurerSegment','InsurerNumber',
                    'InsuredCardSymbol','InsuredIdentificationNumber',
                    'kouhi1','kouhi_jyu1','kouhi2','kouhi_jyu2',
                    'kyufu','institution']

        df_nikkei = self.df_nikkei[selected].copy()
        df_nikkei = df_nikkei[~df_nikkei.duplicated()].copy()
        # merged with Knajya database on id
        df_nikkei = pd.merge(df_nikkei, self.df_kanjya, on=["id"], how="left")

        #print(df_nikkei.columns)
        #df_nikkei = df_nikkei[df_nikkei.id > 90009358].sort_values('id', ascending=False).copy()
        #selectednew=['chozai','id','Name_x','birth_x','InsurerSegment','InsurerNumber',
        #            'InsuredCardSymbol','InsuredIdentificationNumber',
        #            'kouhi1','kouhi_jyu1','kouhi2','kouhi_jyu2',
        #            'kyufu','institution']
        #df_nikkei[selectednew].columns = selected
        #print(df_nikkei.head(10))
        df_merged = pd.merge(df_nikkei,self.df_shikaku,on=["birth", "Name"], how="left")        
        df_merged_birth_hoken = pd.merge(df_nikkei,self.df_shikaku,on=["birth", "InsurerNumber", 
                                'InsuredCardSymbol','InsuredIdentificationNumber'] )

        name_mask = (df_merged_birth_hoken.Name_x != df_merged_birth_hoken.Name_y) #& \
                    #(df_merged_birth_hoken.Name_y.apply(lambda x:len(str(x))) != 0 ) 
        
        df_merged_birth_hoken = df_merged_birth_hoken[name_mask ].copy()

        self.toCsv(df_merged_birth_hoken,"NameDiff.csv")

        # 保険番号、記号、番号、性別　確認
        masks = (df_merged.InsurerNumber_x == df_merged.InsurerNumber_y)  &  \
            (df_merged.InsuredCardSymbol_x == df_merged.InsuredCardSymbol_y )  &   \
             (df_merged.InsuredIdentificationNumber_x == df_merged.InsuredIdentificationNumber_y) # & \
                # (df_merged.sex1 == df_merged.sex2)

        #masks_name = (df_merged_name.InsurerNumber_x == df_merged_name.InsurerNumber_y)  &  \
        #    (df_merged_name.InsuredCardSymbol_x == df_merged_name.InsuredCardSymbol_y )  &   \
        #     (df_merged_name.InsuredIdentificationNumber_x == df_merged_name.InsuredIdentificationNumber_y) & \
        #         (df_merged_name.sex1 == df_merged_name.sex2)

        #print("** file output shikaku with NIKKEI confirmed.....")
        #print(df_merged[masks])
        #self.toCsv(df_merged[masks],"NIKKEI_shikaku_confirmed.csv")
        


        uncheck_list = []
        idx=0

        # Sex check
        df_merged_check = df_merged[masks]
        sex_mask = (df_merged_check.sex1 == df_merged_check.sex2) 
        df_merged_sex_check = df_merged_check[~sex_mask].copy()
        df_merged_sex_check = df_merged_sex_check.insert(loc=idx, column="flag", value=["男女違い"])
        print("sex check.....")
        if df_merged_sex_check != None:
            uncheck_list.append(df_merged_sex_check)        
        else:
            print(" No Sex difference ")

        ## owner family check

        df_merged_check = df_merged[masks]
        owner_family = df_merged_check.InsurerSegment.apply(lambda x:x[2:]).isin( ["本人","家族"])
        df_merged_check = df_merged_check[owner_family].copy()
        yours_masks = (df_merged_check.InsurerSegment.apply(lambda x:x[2:]) != df_merged_check.yours) & \
                                    (df_merged_check.yours != "")
        df_unchecked = df_merged_check[yours_masks]
        df_unchecked = df_unchecked.insert(loc=idx, column="flag", value=["性別違い"])
        print("family check....")
        if df_unchecked != None:
            uncheck_list.append(df_unchecked)
        else:
            print(" No family difference ")

        # PaymentRatio check
        df_pay_check = df_merged[masks]
        pay_check = df_pay_check.InsurerSegment.apply(lambda x:x[:2]).isin( ["社保"])
        pay_check_older = df_pay_check.InsurerSegment.apply(lambda x:x[:2]).isin( ["後期"])

        print(sum(pay_check))

        print(df_merged.head(5))
        print(df_merged.tail(5))
        

        
        df_pay_check_corp = df_pay_check[pay_check]
        df_unchecked_corp = df_pay_check_corp[df_pay_check_corp.kyufu != 70]
        df_unchecked_corp = df_unchecked_corp.insert(loc=idx, column="flag", value=["社保給付割合相違"])
        print("Corp Insurance rate check....")
        if df_unchecked_corp != None:
            uncheck_list.append(df_unchecked_corp)
        else:
            print(" No difference ")

        df_pay_check_nat = df_pay_check[pay_check_older]       
        df_unchecked_nat = df_pay_check_nat[ (df_pay_check_nat.kyufu + df_pay_check_nat.ratio) != 100 ]
        df_unchecked_nat = df_unchecked_nat.insert(loc=idx, column="flag", value=["国保給付割合相違"])
        print("Domestical Insurance rate check....")
        if df_unchecked_nat != None:
            uncheck_list.append(df_unchecked_nat)
        else:
            print(" No difference ")
        #print(df_merged[masks])

        if len(uncheck_list) > 0:
            _df_unchecked = pd.concat(uncheck_list,axis=0)
            self.toCsv(_df_unchecked,"日計保険項目相違.csv")
        else:
            print("*** No report 日計保険相違 ***")


        self.df_merged_unmatched = df_merged[~masks].copy()
        self.df_merged_unmatched.drop_duplicates(inplace=True, subset=["Name"])
        self.toCsv(self.df_merged_unmatched,"日計資格確認端末未確認データ.csv")


        #print("** NON confirmed file output shikaku with NIKKEI by Name.....")
        #print(df_merged[masks])
        #self.toCsv(df_merged_name[~masks_name],"日計未確認_byName.csv")

        #selected_df = df_merged_name[~masks_name].copy()
        #print(selected_df.columns  )
        #target = ["Name","birth_x","InsurerNumber_x", "InsuredCardSymbol_x", "InsuredIdentificationNumber_x"]

        #selected_df = selected_df[target].copy()
        #selected_df.columns = ["Name","birth","InsurerNumber", "InsuredCardSymbol", "InsuredIdentificationNumber"]
        #df_merged_name = pd.merge(selected_df,self.df_shikaku,on=["InsurerNumber", "InsuredCardSymbol", "InsuredIdentificationNumber"], how="left")
        #self.toCsv(df_merged_name,"日計未確認_byName.csv")


        #self.df_shikaku.InsurerNumber 


    def toCsv(self, df, filename):

        filename = os.path.join(self.output_dir,filename)
        df.to_csv(filename, index=False, encoding='cp932', errors='replace')


## Main 

In [35]:
myObject = HokenCheckClass()
myObject.merge()

['ÄæèièmöFùÜù≡_20220304084235694.csv']
change current directory --> L:\epson_pxm840f\Shikakku
## delte unzipped shikaku file... L:\epson_pxm840f\Shikakku\資格確認履歴_20220304084235694.csv


In [38]:
display(myObject.df_merged_birth_hoken2.head(20))

,id,Name_x,birth,InsurerSegment,InsurerNumber,InsuredCardSymbol,InsuredIdentificationNumber,kouhi1,kouhi_jyu1,kouhi2,...,korei_issue,korei_start,korei_end,korei_ratio,gendo_kubun,limit_certification,limit_segment,limit_issue,limit_start,limit_end
0,90009391,中島 近夫,1964/06/14,社保本人,01130012,26010838,217,,,,...,,,,,0.0,,,,,
1,90009392,大嶋 孝聡,1984/01/12,社保本人,06139166,457,1115,,,,...,,,,,0.0,,,,,
2,90009390,中島 渉,2002/03/25,社保家族,01130012,26010838,217,,,,...,,,,,0.0,,,,,
3,90000653,星野 武,1952/10/07,国保本人,138198,19-25,5342,,,,...,,,,,0.0,,,,,
4,90009335,石井 規弘,1957/07/26,国保本人,138198,19-09,0732,,,,...,,,,,0.0,,,,,
5,90008169,石渡 敏之,1960/10/26,国保家族,138198,19-15,0277,,,,...,,,,,0.0,,,,,
6,90000166,田中 和子,1948/04/27,国保（70歳以上8割）,138198,19-60,4454,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,90007224,小森 洋史,1950/04/07,国保（70歳以上8割）福祉,138198,19-21,6438,82137530,,,...,20200801.0,20200801.0,20220731.0,20.0,0.0,,,,,
8,90009393,佐古田 彰,1958/07/31,社保本人,01130012,45474212,6,,,,...,,,,,0.0,,,,,
9,90006947,一瀬 浩道,1952/08/25,社保本人,01130012,45011601,1,,,,...,,,,,0.0,,,,,


In [47]:
my_df = myObject.df_merged_birth_hoken2
myObject.toCsv(my_df,"test.csv")

In [48]:
mask = myObject.df_merged_birth_hoken2.isnull().validation
myObject.df_merged_birth_hoken2[mask]


,id,Name_x,birth,InsurerSegment,InsurerNumber,InsuredCardSymbol,InsuredIdentificationNumber,kouhi1,kouhi_jyu1,kouhi2,...,korei_issue,korei_start,korei_end,korei_ratio,gendo_kubun,limit_certification,limit_segment,limit_issue,limit_start,limit_end
6,90000166,田中 和子,1948/04/27,国保（70歳以上8割）,138198,19-60,4454,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,90003450,田上 義和,1971/02/05,社保本人公費,01130012,45321517,7,38116018,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,90007086,小池 正美,1987/04/30,国保本人福祉,138198,19-35,9637,81137192,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,90006447,内野 博子,1949/08/31,社保（70歳以上8割）,01130012,45190431,2,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,90000360,鈴木 昌代,1953/12/22,社保家族,06135776,5000,6390,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,90002038,小堺 正明,1958/06/11,国保本人,138198,19-40,7441,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,90009331,大久保 貴弘,1971/10/03,社保本人,07130107,自陸練馬駐,30-16024,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,90008416,正木 由美子,1958/03/05,国保本人,120568,56,10072,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,90007370,中山 雅稔,1972/12/19,国保本人,133140,79-047,348,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,90004785,神谷 たみ子,1948/01/01,国保（70歳以上8割）,138198,19-36,4582,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
